In [ ]:
%matplotlib inline

## Import Python packages

- Pretty common stuff: re, sys, os, math, copy, numpy
- cma for optimisation (minimisation of an objective function)
- gvxrPython3 to simulate X-ray images
- imageio to generate a GIF file
- matplotlib to plot graphs and display images
- scikit-image to compare images, and filter images
- scikit-learn to compute the RMSE
- scipy to resample a 1D signal
- tifffile to load and save TIFF images in Float32
- SimpleITK to load the medical DICOM file
- spekpy to generate a beam spectrum

In [ ]:
import numpy as np # Who does not use Numpy?
import re, sys, os, math, copy
from scipy import signal

import cma
from skimage.util import compare_images
from skimage.filters import gaussian
from skimage.filters import laplace
from tifffile import imsave, imread
from sklearn.metrics import mean_squared_error

import matplotlib
# old_backend =  matplotlib.get_backend() 
# matplotlib.use("Agg")  # Prevent showing stuff

from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt # Plotting
from matplotlib.colors import LogNorm # Look up table
from matplotlib.colors import PowerNorm # Look up table
import matplotlib.colors as mcolors

font = {'family' : 'serif',
        #'weight' : 'bold',
         'size'   : 22
       }
matplotlib.rc('font', **font)
matplotlib.rc('text', usetex=True)

import imageio # Save a GIF animation

import SimpleITK as sitk # Load a DICOM file
import spekpy as sp # Create the beam spectrum
import gvxrPython3 as gvxr

## Read the DICOM file

In [ ]:
reader = sitk.ImageFileReader()
reader.SetImageIO("GDCMImageIO")
reader.SetFileName("data/DX000000")
reader.LoadPrivateTagsOn()
reader.ReadImageInformation()    
volume = reader.Execute()
real_image = sitk.GetArrayFromImage(volume)[0]

In [ ]:
# meta_data_keys = volume.GetMetaDataKeys()

# print("DICOM fields:")
# for key in meta_data_keys:
#     print(key, volume.GetMetaData(key))

In [ ]:
# def cleanTags(raw_string):
#     regular_expression = re.compile('<.*?>')
#     clean_text = re.sub(regular_expression, '', raw_string)
#     return clean_text


In [ ]:



# field = volume.GetMetaData("0033|1022")

# for item in field.split("\n"):
#     if "KV" in item:
#         kv = float(cleanTags(item))
        

# print(kv)


## Extract the image size and pixel spacing from the DICOM file to set the X-ray detector parameters for the simulation

In [ ]:
spacing = volume.GetSpacing()[0:2]
size = volume.GetSize()[0:2]

Extract the kVp from the DICOM file to generate the beam spectrum

In [ ]:
kVp = float(volume.GetMetaData("0018|0060"))
print("Peak kilo voltage output of the x-ray generator used: ", kVp)

## Display the experimental radiograph from the DICOM file

In [ ]:
plt.figure(figsize= (20,10))
xrange=range(real_image.shape[1])
yrange=range(real_image.shape[0])

plt.xlabel("Pixel position\n(in mm)")
plt.ylabel("Pixel position\n(in mm)")
plt.title("X-ray image\nkVp:" + str(kVp))
plt.imshow(real_image, cmap="gray", 
           vmin=3934, vmax = 11045,
           extent=[0,(real_image.shape[1]-1)*spacing[0],0,(real_image.shape[0]-1)*spacing[1]])
plt.colorbar(orientation='vertical')

plt.savefig('plots/experimental_image.pdf')
plt.savefig('plots/experimental_image.png')

## Crop it to remove the text

In [ ]:
plt.figure(figsize= (20,10))

roi = [179, 2300, 0, 2848]
roi_real_image = real_image[179:2300,0:2848]

xrange=range(roi_real_image.shape[1])
yrange=range(roi_real_image.shape[0])

plt.xlabel("Pixel position\n(in mm)")
plt.ylabel("Pixel position\n(in mm)")
plt.title("X-ray image\nkVp:" + str(kVp))
plt.imshow(roi_real_image, cmap="gray", 
           vmin=3934, vmax = 11045,
           extent=[roi[2] * spacing[0], (roi[3] -1) *spacing[0], roi[0]*spacing[0],(roi[1] - 1)*spacing[0]])
plt.colorbar(orientation='vertical')

plt.savefig('plots/experimental_image-cropped.pdf')
plt.savefig('plots/experimental_image-cropped.png')

## Set the experimental parameters (e.g. source and detector positions, etc.)

In [ ]:
# source_detector_distance = float(volume.GetMetaData("0018|1110"))
# print("Distance Source to Detector: ", kVp)

In [ ]:
# source_object_distance = float(volume.GetMetaData("0018|1111"))
# print("Distance Source to Patient: ", source_object_distance)

In [ ]:
source_detector_distance_in_cm = 130  # See email Mon 05/07/2021 15:29
block_width_in_cm = 32
block_height_in_cm = 27
block_thickness_in_cm = 4

window_size =  [800, 450]
source_position = [0.0, 0.0, source_detector_distance_in_cm - block_thickness_in_cm / 2, "cm"]
detector_position = [0.0, 0.0, - block_thickness_in_cm / 2, "cm"]
detector_up = [0, 1, 0]

## Initialise the simulation engine

In [ ]:
# Create an OpenGL context
print("Create an OpenGL context:",
    str(window_size[0]) + "x" + str(window_size[1])
);
gvxr.createOpenGLContext();
gvxr.setWindowSize(
    window_size[0],
    window_size[1]
);


## Function to create a PMMA block

In [ ]:
def createBlock(x, y, z, r, h):
    # Remove all the geometries from the whole scenegraph
    gvxr.removePolygonMeshesFromSceneGraph()

    # Make a cube
    gvxr.makeCube("PMMA block", 1.0, "mm")

    # Translation vector
    gvxr.translateNode("PMMA block", x, y, z, "cm")

    # Rotation angle
    gvxr.rotateNode("PMMA block", r, 0, 0, 1)

    # Scaling factors
    gvxr.scaleNode("PMMA block", block_width_in_cm * 10, h * 10, block_thickness_in_cm * 10);

    # Apply the transformation matrix so that we can save the corresponding STL file
    gvxr.applyCurrentLocalTransformation("PMMA block");

    # Set the matrix's material properties
    gvxr.setCompound("PMMA block", "C5O2H8")
    gvxr.setDensity("PMMA block", 1.18, "g/cm3")

    # Add the matrix to the X-ray renderer
    gvxr.addPolygonMeshAsInnerSurface("PMMA block")

In [ ]:
# Initial guess
x = y = 0
z = 0
r = 0
createBlock(x, y, z, r, block_height_in_cm)

## Set the source position

In [ ]:
# Set up the beam
print("Set up the beam")
print("\tSource position:", source_position)
gvxr.setSourcePosition(
    source_position[0],
    source_position[1],
    source_position[2],
    source_position[3]
);

gvxr.usePointSource();

focal_spot_size = volume.GetMetaData("0018|1413")
# gvxr.setFocalSpot(focal_spot_size, focal_spot_size, focal_spot_size, "mm");
print("size of focal spot (in mm):", focal_spot_size)

# Get the spectrum from the DICOM file

In [ ]:
spectrum = {};
filter_material = "Al"      # See email Mon 05/07/2021 15:29
filter_thickness_in_mm = 3  # See email Mon 05/07/2021 15:29

s = sp.Spek(kvp=kVp)
s.filter(filter_material, filter_thickness_in_mm) # Filter by 3 mm of Al
unit = "keV"
k, f = s.get_spectrum(edges=True) # Get the spectrum

min_energy = sys.float_info.max
max_energy = -sys.float_info.max

for energy, count in zip(k, f):
    count = round(count)

    if count > 0:

        max_energy = max(max_energy, energy)
        min_energy = min(min_energy, energy)

        if energy in spectrum.keys():
            spectrum[energy] += count
        else:
            spectrum[energy] = count

Reformat the data

In [ ]:
# get the integral nb of photons
nbphotons=0.
energy1 = -1.
energy2 = -1.

for energy in spectrum.keys():
    
    if energy1<0:
        energy1 = float(energy)
    elif energy2<0:
        energy2 = float(energy)
    nbphotons += float(spectrum[energy])
sampling = (energy2-energy1)

# get spectrum
data = []
for energy in spectrum.keys():
    source = [float(energy),float(spectrum[energy])/(nbphotons*sampling)]
    data.append(source)

# plot spectrum
plt.figure(figsize= (20,10))

data_array = np.array(data)

energies, counts = data_array.T


Resample the data

In [ ]:
number_of_energy_bins = 30
count_set = signal.resample(counts, number_of_energy_bins)
energy_set = np.linspace(energies.min(), energies.max(), number_of_energy_bins, endpoint=True)

In [ ]:
plt.figure(figsize= (20,10))
plt.plot(energies, counts, label="Original")
plt.plot(energy_set, count_set, "o", label="Resampled")
plt.xlabel('energy in keV')
plt.ylabel('probability distribution of photons per keV')
plt.legend()
plt.savefig("plots/spectrum.pdf")

## Load the beam spectrum in the simulator

In [ ]:
# for energy in spectrum.keys():
#     count = spectrum[energy]
#     gvxr.addEnergyBinToSpectrum(energy, unit, count);

for energy, count in zip(energy_set, count_set):
    gvxr.addEnergyBinToSpectrum(energy, unit, count);

## Set the X-ray detector

In [ ]:
# Set up the detector
print("Set up the detector");
print("\tDetector position:", detector_position)
gvxr.setDetectorPosition(
    detector_position[0],
    detector_position[1],
    detector_position[2],
    detector_position[3]
);

print("\tDetector up vector:", detector_up)
gvxr.setDetectorUpVector(
    detector_up[0],
    detector_up[1],
    detector_up[2]
);

In [ ]:
print("\tDetector number of pixels:", size)
gvxr.setDetectorNumberOfPixels(
    size[0],
    size[1]
);

print("\tPixel spacing:", spacing)
gvxr.setDetectorPixelSize(
    spacing[0],
    spacing[1],
    "mm"
);

## Take a screenshot of the 3D environment

In [ ]:
gvxr.displayScene()

gvxr.useLighing()
gvxr.useWireframe()
gvxr.setSceneRotationMatrix([0.43813619017601013, 0.09238918125629425, -0.8941444158554077, 0.0,
                             0.06627026945352554, 0.9886708855628967, 0.13463231921195984, 0.0,
                             0.8964602947235107, -0.11824299395084381, 0.4270564019680023, 0.0,
                             0.0, 0.0, 0.0, 1.0])
gvxr.setZoom(1639.6787109375)

gvxr.displayScene()

In [ ]:
screenshot = gvxr.takeScreenshot()

In [ ]:
plt.figure(figsize= (10,10))
plt.title("Screenshot")
plt.imshow(screenshot)
plt.axis('off')

plt.tight_layout()

plt.savefig('plots/screenshot-beam-off.pdf')
plt.savefig('plots/screenshot-beam-off.png')

# Optimise the position and orientation of the PMMA block, and refine the SDD

In [ ]:
def standardisation(image):
    return (image - image.mean()) / image.std();

In [ ]:
def zncc(i1, i2):
    return (np.mean(np.multiply(i1, i2))) / 2.0;

In [ ]:
def objectiveFunction(parameters):
    
    global best_fitness
    global best_fitness_id
    global fitness_function_call_id
    global evolution_zncc
    global evolution_parameters
    
    # Retrieve the parameters
    x, y, z, r, h, SDD = parameters
    
    # Update the source position
    source_position = [0.0, 0.0, SDD - block_thickness_in_cm / 2, "cm"]
    gvxr.setSourcePosition(
        source_position[0],
        source_position[1],
        source_position[2],
        source_position[3]
    );

    # Update the block geometry
    createBlock(x, y, z, r, h)
    
    # Compute an X-ray image
    x_ray_image = np.array(gvxr.computeXRayImage())
    
    # Compute the negative image as it is the case for the real image
    x_ray_image *= -1.
    
    # Crop the image
    x_ray_image = x_ray_image[179:2300,0:2848]

    # Zero-mean, unit-variance normalistion
    x_ray_image = standardisation(x_ray_image)
    
    # Return the objective
    objective = math.sqrt(mean_squared_error(roi_real_image, x_ray_image))

    # The block below is not necessary for the registration.
    # It is used to save the data to create animations.
    if best_fitness > objective:
        
        gvxr.displayScene()
        screenshot = (255 * np.array(gvxr.takeScreenshot())).astype(np.uint8)

        gvxr.saveSTLfile("PMMA block", "outputs/PMMA_block_" + str(best_fitness_id) + ".stl")
        np.savetxt("outputs/PMMA_block_" + str(best_fitness_id) + ".dat", [x, y, z, r, h, SDD], header='x,y,z,r,h,SDD')
        imsave("outputs/xray_" + str(best_fitness_id) + ".tif", x_ray_image.astype(np.single))
        imsave("outputs/screenshot_" + str(best_fitness_id) + ".tif", screenshot)
    
        zncc_value = zncc(roi_real_image, x_ray_image)
        evolution_zncc.append([fitness_function_call_id, zncc_value])
        evolution_parameters.append([fitness_function_call_id, x, y, z, r, h, SDD])
        
        best_fitness = objective
        best_fitness_id += 1
    
    fitness_function_call_id += 1
    
    return objective

In [ ]:
# Zero-mean, unit-variance normalistion
roi_real_image = standardisation(roi_real_image)
imsave("outputs/roi_real_image.tif", roi_real_image)

In [ ]:
# The registration has already been performed. Load the results.
if os.path.isfile("outputs/PMMA_block.dat") and os.path.isfile("outputs/evolution_zncc.dat") and os.path.isfile("outputs/evolution_parameters.dat"):
    
    temp = np.loadtxt("outputs/PMMA_block.dat")
    x = temp[0]
    y = temp[1]
    z = temp[2]
    r = temp[3]
    block_height_in_cm = temp[4]
    source_detector_distance_in_cm = temp[5]
    
    # Update the source position
    source_position = [0.0, 0.0, source_detector_distance_in_cm - block_thickness_in_cm / 2, "cm"]
    gvxr.setSourcePosition(
        source_position[0],
        source_position[1],
        source_position[2],
        source_position[3]
    );
    
    # Update the block geometry
    createBlock(x, y, z, r, block_height_in_cm)

    evolution_zncc = np.loadtxt("outputs/evolution_zncc.dat")
    evolution_parameters = np.loadtxt("outputs/evolution_parameters.dat")

else:
    # Optimise
    opts = cma.CMAOptions()
    opts.set('tolfun', 1e-2)
    opts['tolx'] = 1e-2
    opts['bounds'] = [[-20, -20, detector_position[2] + block_thickness_in_cm / 2, -90, block_height_in_cm * 0.80, source_detector_distance_in_cm * 0.80],
                      [20, 20, source_position[2] - block_thickness_in_cm / 2, 90, block_height_in_cm * 1.20, source_detector_distance_in_cm * 1.20]]
    opts['CMA_stds'] = []

    for min_val, max_val in zip(opts['bounds'][0], opts['bounds'][1]):
        opts['CMA_stds'].append(abs(max_val - min_val) * 0.05)

    best_fitness = sys.float_info.max;
    best_fitness_id = 0;
    fitness_function_call_id = 0
    evolution_zncc = []
    evolution_parameters = []
        
    # Optimise
    res = cma.fmin(objectiveFunction,
              [x, y, z, r, block_height_in_cm, source_detector_distance_in_cm],
              0.5,
              opts,
              restarts=1) 
    x, y, z, r, block_height_in_cm, source_detector_distance_in_cm = res[0]
    
#     es = cma.CMAEvolutionStrategy([x, y, z, r, source_detector_distance_in_cm], 0.5, opts)
#     es.optimize(objectiveFunction)
#     x, y, z, r, source_detector_distance_in_cm = es.result.xbest
#     del es;

    # Save the parameters
    np.savetxt("outputs/PMMA_block.dat", [x, y, z, r, block_height_in_cm, source_detector_distance_in_cm], header='x,y,z,r,h,SDD')

    # Update the source position
    source_position = [0.0, 0.0, source_detector_distance_in_cm - block_thickness_in_cm / 2, "cm"]
    gvxr.setSourcePosition(
        source_position[0],
        source_position[1],
        source_position[2],
        source_position[3]
    );
    
    # Update the block geometry
    createBlock(x, y, z, r, block_height_in_cm)
    gvxr.saveSTLfile("PMMA block", "outputs/PMMA_block_" + str(best_fitness_id) + ".stl")

    evolution_zncc = np.array(evolution_zncc)
    np.savetxt("outputs/evolution_zncc.dat", evolution_zncc, header='t,ZNCC')

    evolution_parameters = np.array(evolution_parameters)
    np.savetxt("outputs/evolution_parameters.dat", evolution_parameters, header='t,x,y,z,r,h,SDD')

In [ ]:
print("SDD (in cm):", source_detector_distance_in_cm)
print("Block centre (in cm):", x, y, z)
print("Block size (in cm):", block_width_in_cm, block_height_in_cm, block_thickness_in_cm)
print("Rotation angle (in degrees):", r)

In [ ]:
evolution_parameters = np.array(evolution_parameters)

plt.figure(figsize= (20,10))
plt.title("Evolution of the PMMA block centre of mass along the three carthesian coordinates")
plt.plot(evolution_parameters[:,0], evolution_parameters[:,1], label="x")
plt.plot(evolution_parameters[:,0], evolution_parameters[:,2], label="y")
plt.plot(evolution_parameters[:,0], evolution_parameters[:,3], label="z")
plt.xlabel("Timeline\n(in number of fitness function calls)")
plt.ylabel("Position (in cm)")
plt.legend()
plt.savefig('plots/evolution_centre_of_mass.pdf')
plt.savefig('plots/evolution_centre_of_mass.png')

plt.figure(figsize= (20,10))
plt.title("Evolution of the rotation angle of the PMMA block")
plt.plot(evolution_parameters[:,0], evolution_parameters[:,4], label="angle")
plt.xlabel("Timeline\n(in number of fitness function calls)")
plt.ylabel("Rotation angle (in degrees)")
plt.savefig('plots/evolution_centre_of_mass.pdf')
plt.savefig('plots/evolution_centre_of_mass.png')

plt.figure(figsize= (20,10))
plt.title("Evolution of the height of the PMMA block")
plt.plot(evolution_parameters[:,0], evolution_parameters[:,5], label="angle")
plt.xlabel("Timeline\n(in number of fitness function calls)")
plt.ylabel("Height (in cm)")
plt.savefig('plots/evolution_height.pdf')
plt.savefig('plots/evolution_height.png')

plt.figure(figsize= (20,10))
plt.title("Evolution of the source-detector distance")
plt.plot(evolution_parameters[:,0], evolution_parameters[:,6])
plt.xlabel("Timeline\n(in number of fitness function calls)")
plt.ylabel("Distance (in mm)")
plt.savefig('plots/evolution_SDD.pdf')
plt.savefig('plots/evolution_SDD.png')

In [ ]:
# Compute an X-ray image
raw_x_ray_image = np.array(gvxr.computeXRayImage())

gvxr.displayScene()

# Compute the negative image as it is the case for the real image
x_ray_image = raw_x_ray_image * -1.

# Crop the image
x_ray_image = x_ray_image[179:2300,0:2848]

# Zero-mean, unit-variance normalistion
x_ray_image = standardisation(x_ray_image)

## Plot the evolution of the quality of the simulated image

In [ ]:
plt.figure(figsize= (20,10))
plt.title("Evolution of the ZNCC between the experimental and simulated images")
plt.plot(evolution_zncc[:,0], 100.0 * evolution_zncc[:,1], "go-")
plt.xlabel("Timeline\n(in number of fitness function calls)")
plt.ylabel("ZNCC\n(in \%)")
plt.savefig('plots/evolution_ZNCC1.pdf')
plt.savefig('plots/evolution_ZNCC1.png')

In [ ]:
# gvxr.renderLoop()
# print(gvxr.getZoom())
# print(gvxr.getSceneRotationMatrix())

In [ ]:
def compareImages(real_image, x_ray_image, colorbar=True, rows = 1, cols = 3):
    comp_equalized = compare_images(real_image, x_ray_image, method='checkerboard');
    
#     plt.figure(figsize= (20,10))

    # plt.suptitle("Image simulated using gVirtualXRay without the energy response of the detector", y=1.02)

    plt.subplot(rows, cols, 1)
    plt.imshow(real_image, cmap="gray", vmin=-1, vmax=1,
               extent=[0,(real_image.shape[1]-1)*spacing[0],0,(real_image.shape[0]-1)*spacing[1]])
    if colorbar:
        plt.colorbar(orientation='horizontal')
    plt.title("Experimental image")
    plt.xlabel("Pixel position\n(in mm)")
    plt.ylabel("Pixel position\n(in mm)")

    plt.subplot(rows, cols, 2)
    plt.imshow(x_ray_image, cmap="gray", vmin=-1, vmax=1,
               extent=[0,(real_image.shape[1]-1)*spacing[0],0,(real_image.shape[0]-1)*spacing[1]])
    if colorbar:
        plt.colorbar(orientation='horizontal')
    plt.title("Simulated image")
    plt.xlabel("Pixel position\n(in mm)")
    plt.ylabel("Pixel position\n(in mm)")

    plt.subplot(rows, cols, 3)
    plt.imshow(comp_equalized, cmap="gray", vmin=-1, vmax=1,
               extent=[0,(real_image.shape[1]-1)*spacing[0],0,(real_image.shape[0]-1)*spacing[1]])
    if colorbar:
        plt.colorbar(orientation='horizontal')
    plt.title("Checkerboard comparison")
    plt.xlabel("Pixel position\n(in mm)")
    plt.ylabel("Pixel position\n(in mm)")

    plt.tight_layout()

In [ ]:
plt.figure(figsize= (20,10))
compareImages(roi_real_image, x_ray_image)
plt.savefig('plots/compare_images1.pdf')
plt.savefig('plots/compare_images1.png')

In [ ]:
def drawHorizontalProfiles(real_image, x_ray_image):

    horizontal_profile_real_image = real_image[real_image.shape[1] // 2]
    horizontal_profile_simulated_image = x_ray_image[x_ray_image.shape[1] // 2]

    x_val = np.linspace(0.0,
                        spacing[0] * real_image.shape[1],
                        real_image.shape[1], endpoint=True)

    plt.plot(x_val, horizontal_profile_real_image, label="Experimental image")
    plt.plot(x_val, horizontal_profile_simulated_image, label="Simulated image")

    plt.xlabel("Pixel position\n(in mm)")
    plt.ylabel("Relative intensity")

In [ ]:
def drawVerticalProfiles(real_image, x_ray_image):
    vertical_profile_real_image = real_image[:,real_image.shape[0] // 2]
    vertical_profile_simulated_image = x_ray_image[:,x_ray_image.shape[0] // 2]

    x_val = np.linspace(0.0,
                        spacing[1] * real_image.shape[0],
                        real_image.shape[0], endpoint=True)

    plt.plot(x_val, vertical_profile_real_image, label="Experimental image")
    plt.plot(x_val, vertical_profile_simulated_image, label="Simulated image")

In [ ]:
plt.figure(figsize= (20,10))
drawHorizontalProfiles(roi_real_image, x_ray_image)
plt.legend()
plt.savefig('plots/compare_horizontal_profile1.pdf')
plt.savefig('plots/compare_horizontal_profile1.png')

In [ ]:
plt.figure(figsize= (20,10))
drawVerticalProfiles(roi_real_image, x_ray_image)
plt.legend()
plt.savefig('plots/compare_vertical_profile1.pdf')
plt.savefig('plots/compare_vertical_profile1.png')

In [ ]:
def createAnimation(evolution_parameters, real_image):
    # Create the GIF file
    with imageio.get_writer("plots/evolution.gif", mode='I') as writer:

        # Store the PNG filenames
        png_filename_set = [];

        # Process all the images
        for i, [t, x, y, z, r, w, SDD] in enumerate(evolution_parameters):
            t = int(t)
            
            x_ray_image = imread("outputs/xray_" + str(i) + ".tif")
            screenshot = imread("outputs/screenshot_" + str(i) + ".tif")
            
            # Create the figure
            fig, axs = plt.subplots(nrows=2, ncols=3, figsize= (20,10))
            plt.suptitle("Iteration " + str(t+1) + "/" + str(int(evolution_parameters[-1][0]+1)))
            compareImages(real_image, x_ray_image, False, 2, 3)
      
            plt.subplot(234)
            drawHorizontalProfiles(real_image, x_ray_image)

            plt.subplot(235)
            drawVerticalProfiles(real_image, x_ray_image)
  

            plt.subplot(236)
            plt.imshow(screenshot)

            # Save the figure as a PNG file
            plt.savefig("temp.png")

            # Close the figure
            plt.close()

            # Open the PNG file with imageio and add it to the GIF file
            image = imageio.imread("temp.png")
            writer.append_data(image)

            # Delete the PNG file
            os.remove("temp.png");

        for i in range(15):
            writer.append_data(image)

In [ ]:
if not os.path.exists("plots/evolution.gif"):
    createAnimation(evolution_parameters, roi_real_image)

![GIF animation of the artificial evolution results](plots/evolution.gif)

In [ ]:
def sharpen(image, ksize, alpha, shift, scale):
    details = image - gaussian(image, ksize)    
#     details = laplace(image, ksize)    
    return scale * (shift + image) + alpha * details

In [ ]:
def objectiveFunctionSharpen(parameters):
    
    global best_fitness
    global best_fitness_id
    global fitness_function_call_id
    global sharpened_evolution_zncc
    global sharpened_evolution_parameters

    global raw_x_ray_image
    
    # Retrieve the parameters
    sigma, alpha, shift, scale = parameters
    
    # Compute an X-ray image
    x_ray_image = sharpen(raw_x_ray_image, sigma, alpha, shift, scale)
    
    # Compute the negative image as it is the case for the real image
    x_ray_image *= -1.
    
    # Crop the image
    x_ray_image = x_ray_image[179:2300,0:2848]

    # Zero-mean, unit-variance normalistion
    x_ray_image = standardisation(x_ray_image)
    
    # Return the objective
    objective = math.sqrt(mean_squared_error(roi_real_image, x_ray_image))

    # The block below is not necessary for the registration.
    # It is used to save the data to create animations.
    if best_fitness > objective:
        
        imsave("outputs/sharpened_xray_" + str(best_fitness_id) + ".tif", x_ray_image.astype(np.single))
    
        zncc_value = zncc(roi_real_image, x_ray_image)
        sharpened_evolution_zncc.append([fitness_function_call_id, zncc_value])
        sharpened_evolution_parameters.append([fitness_function_call_id, sigma, alpha, shift, scale])
        
        best_fitness = objective
        best_fitness_id += 1
    
    fitness_function_call_id += 1
    
    return objective

In [ ]:
old_zncc = evolution_zncc[-1][1]

In [ ]:
# The registration has already been performed. Load the results.
if os.path.isfile("outputs/sharpen.dat") and os.path.isfile("outputs/sharpen_evolution_zncc.dat") and os.path.isfile("outputs/sharpen_evolution_parameters.dat"):

    temp = np.loadtxt("outputs/sharpen.dat")
    sigma = temp[0]
    alpha = temp[1]
    shift = temp[2]
    scale = temp[3]
    
    sharpened_evolution_zncc = np.loadtxt("outputs/sharpen_evolution_zncc.dat")
    sharpened_evolution_parameters = np.loadtxt("outputs/sharpen_evolution_parameters.dat")

else:
    # Optimise
    opts = cma.CMAOptions()
    opts.set('tolfun', 1e-3)
    opts['tolx'] = 1e-3
    opts['bounds'] = [[3, 0, -5, 0],
                      [200, 15, 5, 2]]
    opts['CMA_stds'] = []

    for min_val, max_val in zip(opts['bounds'][0], opts['bounds'][1]):
        opts['CMA_stds'].append(abs(max_val - min_val) * 0.05)

    best_fitness = sys.float_info.max;
    best_fitness_id = 0;
    fitness_function_call_id = 0
    sharpened_evolution_zncc = []
    sharpened_evolution_parameters = []
    
    sigma = 100
    alpha = 2.5
    shift = 0
    scale = 1
    
    # Optimise
    es = cma.CMAEvolutionStrategy([sigma, alpha, shift, scale], 0.5, opts)
    es.optimize(objectiveFunctionSharpen)

    # Save the parameters
    sigma, alpha, shift, scale = es.result.xbest
    np.savetxt("outputs/sharpen.dat", [sigma, alpha, shift, scale], header='sigma,alpha,shift,scale')

    sharpened_evolution_zncc = np.array(sharpened_evolution_zncc)
    np.savetxt("outputs/sharpen_evolution_zncc.dat", sharpened_evolution_zncc, header='t,ZNCC')

    sharpened_evolution_parameters = np.array(sharpened_evolution_parameters)
    np.savetxt("outputs/sharpen_evolution_parameters.dat", sharpened_evolution_parameters, header='t,sigma,alpha')

    # Release memory
    del es;

In [ ]:
new_zncc = sharpened_evolution_zncc[-1][1]

In [ ]:
print("ZNCC before sharpening:", str(100 * old_zncc) + "%")
print("ZNCC after sharpening:", str(100 * new_zncc) + "%")

In [ ]:
print(evolution_zncc[-1,0])

In [ ]:
plt.figure(figsize= (20,10))
plt.title("Evolution of the ZNCC between the experimental and simulated images")
plt.plot(evolution_zncc[:,0], 100 * evolution_zncc[:,1], "go-", label="Optimisation of the geometry")
plt.plot(sharpened_evolution_zncc[:,0] + evolution_zncc[-1,0], 100 * sharpened_evolution_zncc[:,1], "ro-", label="Optimisation of the digital imaging filters")
plt.xlabel("Timeline\n(in number of fitness function calls)")
plt.ylabel("ZNCC\n(in \%)")
plt.legend()
plt.savefig('plots/evolution_ZNCC2.pdf')
plt.savefig('plots/evolution_ZNCC2.png')

In [ ]:
# Compute an X-ray image
x_ray_image = sharpen(raw_x_ray_image, sigma, alpha, shift, scale)

# Compute the negative image as it is the case for the real image
x_ray_image *= -1.

# Crop the image
x_ray_image = x_ray_image[179:2300,0:2848]

# Zero-mean, unit-variance normalistion
x_ray_image = standardisation(x_ray_image)

In [ ]:
plt.figure(figsize= (20,10))
compareImages(roi_real_image, x_ray_image)
plt.savefig('plots/compare_images2.pdf')
plt.savefig('plots/compare_images2.png')

In [ ]:
plt.figure(figsize= (20,10))
drawHorizontalProfiles(roi_real_image, x_ray_image)
plt.savefig('plots/compare_horizontal_profile2.pdf')
plt.savefig('plots/compare_horizontal_profile2.png')

In [ ]:
plt.figure(figsize= (20,10))
drawVerticalProfiles(roi_real_image, x_ray_image)
plt.savefig('plots/compare_vertical_profile2.pdf')
plt.savefig('plots/compare_vertical_profile2.png')

In [ ]:
print(sigma, alpha)


In [ ]:
print(shift, scale)